In [8]:
import os
from dotenv import load_dotenv
load_dotenv('.env', verbose=True)

from typing import Literal, TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, BaseMessage
from langgraph.graph import StateGraph, END, MessageGraph
from langgraph.prebuilt import ToolNode

## Setup

As above, we will first define the tools we want to use. For this simple example, we will use a web search tool. However, it is really easy to create your own tools - see documentation here on how to do that.


We can now wrap these tools in a simple LangGraph ToolNode. This class receives the list of messages (containing tool_calls, calls the tool(s) the LLM has requested to run, and returns the output as new ToolMessage(s).

After we've done this, we should make sure the model knows that it has these tools available to call. We can do this by converting the LangChain tools into the format for OpenAI tool calling using the bind_tools() method.

In [9]:
tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)
model = ChatOpenAI(name='gpt-3.5-turbo', temperature=0)

model = model.bind_tools(tools)



## Define Agent State

This time, we'll use the more general StateGraph. This graph is parameterized by a state object that it passes around to each node. Remember that each node then returns operations to update that state. These operations can either SET specific attributes on the state (e.g. overwrite the existing values) or ADD to the existing attribute. Whether to set or add is denoted by annotating the state object you construct the graph with.

For this example, the state we will track will just be a list of messages. We want each node to just add messages to that list. Therefore, we will use a TypedDict with one key (messages) and annotate it so that we always add to the messages key when updating it using the is always added to with the second parameter (operator.add). (Note: the state can be any type, including pydantic BaseModel's).

在你提到的代码段中，`AgentState` 使用了 Python 的 `TypedDict` 和 `Annotated` 类型，这些都是类型注解工具，用来帮助定义字典的键和相关的类型信息，以及如何处理这些类型的更新。这个代码主要是为了设定一个状态对象，这个状态对象被用于 LangChain 中的状态图（StateGraph）中。下面我会详细解释每个部分的意义和作用：

### TypedDict
`TypedRetypedDict` 是 Python 的一个类型注解工具，它允许你为字典的每个键指定应该有的类型。这是非常有用的，特别是在需要清晰定义字典结构的场景中，比如 API 的数据传输对象（DTOs）或复杂的配置对象。在你的代码中，`TypedDict` 用来定义一个名为 `AgentState` 的字典类型，这个字典有一个键 `messages`。

### Annotated
`Annotated` 是 Python 类型系统的一部分，用于为已存在的类型添加元数据。在你的代码中，`Annotated` 被用来给 `list` 类型附加一个特定的函数 `add_messages`，这个函数定义了当状态更新时应如何处理这个列表。

### add_messages 函数
`add_messages` 函数定义了如何更新 `messages` 这个列表。通常，在不同的节点中，你可能需要添加消息而不是替换现有的消息，所以这个函数确保每次更新状态时，新的消息将被添加到现有的列表中，而不是覆盖它。

### 将它们组合起来
结合使用 `TypedDict` 和 `Annotated` 允许你详细地控制如何更新 `AgentState` 中的 `messages` 列表。每次状态更新时，都会调用 `add_messages` 函数来处理新旧列表的合并，这确保了状态的连续性和数据的完整性。

这种方式在构建需要维护复杂状态的应用程序时非常有用，比如聊天机器人或其他需要追踪对话状态的交互式应用程序。

希望这样的解释可以帮助你更好地理解这些代码的作用！如果你有任何疑问或需要进一步的解释，请随时提问。

In [10]:
def add_messages(left: list, right: list):
    """ Add-don't-overwrite"""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]



## Define the nodes

We now need to define a few different nodes in our graph. In langgraph, a node can be either a regular python function or a runnable.

There are two main nodes we need for this:

The agent: responsible for deciding what (if any) actions to take.
A function to invoke tools: if the agent decides to take an action, this node will then execute that action. We already defined this above.
We will also need to define some edges. Some of these edges may be conditional. The reason they are conditional is that the destination depends on the contents of the graph's State.

The path that is taken is not known until that node is run (the LLM decides). For our use case, we will need one of each type of edge:

Conditional Edge: after the agent is called, we should either:

a. Run tools if the agent said to take an action, OR

b. Finish (respond to the user) if the agent did not ask to run tools

Normal Edge: after the tools are invoked, the graph should always return to the agent to decide what to do next

Let's define the nodes, as well as a function to define the conditional edge to take.

In [11]:
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_msg = messages[-1]

    if last_msg.tool_calls:
        return "tools"

    return "__end__"

def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)

    return {'messages': [response]}



## Define the graph

We can now put it all together and define the graph!

In [12]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node('tools', tool_node)

workflow.set_entry_point('agent')

workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", 'agent')

app = workflow.compile()

## Use it

We can now use it! This now exposes the same interface as all other LangChain runnables. This runnable accepts a list of messages.

In [13]:
inputs = {'messages': [HumanMessage("What's the weather in Nanshan, Shenzhen")]}
app.invoke(inputs)

{'messages': [HumanMessage(content="What's the weather in Nanshan, Shenzhen"),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_NNxFyMnAsP3bIGheLstAaLJN', 'function': {'arguments': '{"query":"weather in Nanshan, Shenzhen"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2c262894-2ca1-45ac-92f1-8d4ec1cecac3-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Nanshan, Shenzhen'}, 'id': 'call_NNxFyMnAsP3bIGheLstAaLJN'}], usage_metadata={'input_tokens': 92, 'output_tokens': 25, 'total_tokens': 117}),
  ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Shenzhen\', \'region\': \'Guangdong\', \'country\': \'China\', \'lat\': 22.53, \'lon\': 114.13, \'tz_id\